# Kelvin-Helmholtz Instability with SPH

The Kelvin-Helmholtz instability is one of the most beautiful and important fluid instabilities in nature. It occurs when there is a velocity difference across the interface between two fluids, creating characteristic "cat's eye" vortical structures.

## Physical Background

The Kelvin-Helmholtz instability arises from the shear flow between two fluid layers moving at different velocities. Small perturbations at the interface grow exponentially, leading to the formation of vortical structures. This instability is seen everywhere from ocean waves to atmospheric phenomena, and even in astrophysical jets.

The growth rate of the instability depends on:
- The velocity difference between the layers
- The density contrast between the fluids
- The wavelength of the perturbation

## What we'll do in this notebook

1. **Generate initial conditions** with two fluid layers at different velocities
2. **Create parameter files** for SWIFT
3. **Run the simulation** (if SWIFT is installed)
4. **Visualize the results** and create movies
5. **Explore parameter variations**

Let's get started!

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import unyt
from scipy.spatial.distance import cdist
import os
import subprocess

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Setting up the Initial Conditions

For the Kelvin-Helmholtz instability, we need to create two layers of fluid with different velocities. We'll set up a 2D simulation in a rectangular box.

In [ ]:
def generate_kelvin_helmholtz_ic(n_particles_x=200, n_particles_y=100, 
                                box_size=[2.0, 1.0], velocity_shear=1.0,
                                density_contrast=2.0, perturbation_amplitude=0.01):
    """
    Generate initial conditions for Kelvin-Helmholtz instability
    
    Parameters:
    -----------
    n_particles_x, n_particles_y : int
        Number of particles in x and y directions
    box_size : list
        Size of the simulation box [x, y]
    velocity_shear : float
        Velocity difference between upper and lower layers
    density_contrast : float
        Density ratio between upper and lower layers
    perturbation_amplitude : float
        Amplitude of initial velocity perturbation
    """
    
    # Create regular grid of particles
    x = np.linspace(0, box_size[0], n_particles_x, endpoint=False)
    y = np.linspace(0, box_size[1], n_particles_y, endpoint=False)
    
    # Add small random offset to avoid perfect grid
    dx = box_size[0] / n_particles_x
    dy = box_size[1] / n_particles_y
    
    xx, yy = np.meshgrid(x + dx/2, y + dy/2, indexing='ij')
    
    # Add small random perturbations to avoid perfect regularity
    xx += np.random.uniform(-0.1*dx, 0.1*dx, xx.shape)
    yy += np.random.uniform(-0.1*dy, 0.1*dy, yy.shape)
    
    # Flatten arrays
    pos_x = xx.flatten()
    pos_y = yy.flatten()
    pos_z = np.zeros_like(pos_x)  # 2D simulation, so z=0
    
    n_particles = len(pos_x)
    print(f"Generated {n_particles} particles")
    
    # Set up velocities with shear flow
    # Lower half: negative velocity, upper half: positive velocity
    vel_x = np.where(pos_y < box_size[1]/2, -velocity_shear/2, velocity_shear/2)
    vel_y = np.zeros_like(pos_x)
    vel_z = np.zeros_like(pos_x)
    
    # Add small perturbations to trigger instability
    # Sinusoidal perturbation in y-velocity
    wavelength = box_size[0] / 4  # 4 wavelengths across the box
    vel_y += perturbation_amplitude * np.sin(2 * np.pi * pos_x / wavelength) * \
             np.exp(-((pos_y - box_size[1]/2) / (box_size[1]/10))**2)
    
    # Set up densities with contrast
    rho = np.where(pos_y < box_size[1]/2, 1.0, 1.0/density_contrast)
    
    # Particle masses (constant for simplicity)
    total_mass = np.sum(rho) * (box_size[0] * box_size[1]) / n_particles
    masses = np.full(n_particles, total_mass / n_particles)
    
    # Internal energy (constant temperature)
    u = np.full(n_particles, 1.0)
    
    # Particle IDs
    ids = np.arange(n_particles, dtype=np.int64)
    
    return {
        'positions': np.column_stack([pos_x, pos_y, pos_z]),
        'velocities': np.column_stack([vel_x, vel_y, vel_z]),
        'masses': masses,
        'densities': rho,
        'internal_energies': u,
        'ids': ids,
        'n_particles': n_particles,
        'box_size': box_size
    }

# Generate the initial conditions
ic_data = generate_kelvin_helmholtz_ic(n_particles_x=150, n_particles_y=75,
                                      box_size=[4.0, 2.0], 
                                      velocity_shear=2.0,
                                      density_contrast=2.0,
                                      perturbation_amplitude=0.1)

print(f"Box size: {ic_data['box_size']}")
print(f"Number of particles: {ic_data['n_particles']}")

## 2. Visualizing the Initial Conditions

Let's look at what we've created:

In [ ]:
# Plot initial conditions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

pos = ic_data['positions']
vel = ic_data['velocities']
rho = ic_data['densities']

# Density field
im1 = axes[0,0].scatter(pos[:,0], pos[:,1], c=rho, s=1, cmap='viridis')
axes[0,0].set_title('Density')
axes[0,0].set_xlabel('x')
axes[0,0].set_ylabel('y')
axes[0,0].set_aspect('equal')
plt.colorbar(im1, ax=axes[0,0])

# X-velocity field
im2 = axes[0,1].scatter(pos[:,0], pos[:,1], c=vel[:,0], s=1, cmap='RdBu_r')
axes[0,1].set_title('X-Velocity')
axes[0,1].set_xlabel('x')
axes[0,1].set_ylabel('y')
axes[0,1].set_aspect('equal')
plt.colorbar(im2, ax=axes[0,1])

# Y-velocity field (perturbations)
im3 = axes[1,0].scatter(pos[:,0], pos[:,1], c=vel[:,1], s=1, cmap='RdBu_r')
axes[1,0].set_title('Y-Velocity (Perturbations)')
axes[1,0].set_xlabel('x')
axes[1,0].set_ylabel('y')
axes[1,0].set_aspect('equal')
plt.colorbar(im3, ax=axes[1,0])

# Velocity magnitude and direction
vel_mag = np.sqrt(vel[:,0]**2 + vel[:,1]**2)
im4 = axes[1,1].scatter(pos[:,0], pos[:,1], c=vel_mag, s=1, cmap='plasma')
# Add some velocity arrows
skip = 20  # Show every 20th particle
axes[1,1].quiver(pos[::skip,0], pos[::skip,1], 
                vel[::skip,0], vel[::skip,1], 
                scale=10, alpha=0.7, color='white', width=0.002)
axes[1,1].set_title('Velocity Magnitude and Direction')
axes[1,1].set_xlabel('x')
axes[1,1].set_ylabel('y')
axes[1,1].set_aspect('equal')
plt.colorbar(im4, ax=axes[1,1])

plt.tight_layout()
plt.show()

## 3. Writing Initial Conditions to HDF5 File

SWIFT reads initial conditions from HDF5 files with a specific format. Let's create this file:

In [ ]:
def write_swift_ic_file(ic_data, filename):
    """
    Write initial conditions to SWIFT-compatible HDF5 file
    """
    
    # Ensure output directory exists
    os.makedirs('../ics', exist_ok=True)
    filepath = f'../ics/{filename}'
    
    with h5py.File(filepath, 'w') as f:
        
        # Header information
        header = f.create_group('Header')
        header.attrs['BoxSize'] = ic_data['box_size'] + [1.0]  # Add z-dimension
        header.attrs['NumPart_ThisFile'] = [ic_data['n_particles'], 0, 0, 0, 0, 0]
        header.attrs['NumPart_Total'] = [ic_data['n_particles'], 0, 0, 0, 0, 0]
        header.attrs['NumPart_Total_HighWord'] = [0, 0, 0, 0, 0, 0]
        header.attrs['MassTable'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        header.attrs['Time'] = 0.0
        header.attrs['Redshift'] = 0.0
        header.attrs['Flag_Sfr'] = 0
        header.attrs['Flag_Feedback'] = 0
        header.attrs['Flag_Cooling'] = 0
        header.attrs['Flag_StellarAge'] = 0
        header.attrs['Flag_Metals'] = 0
        header.attrs['NumFilesPerSnapshot'] = 1
        header.attrs['HubbleParam'] = 1.0
        header.attrs['Omega0'] = 0.0
        header.attrs['OmegaLambda'] = 0.0
        
        # Particle type 0 (gas/SPH particles)
        part0 = f.create_group('PartType0')
        
        # Positions
        coords = ic_data['positions'].copy()
        part0.create_dataset('Coordinates', data=coords)
        
        # Velocities  
        part0.create_dataset('Velocities', data=ic_data['velocities'])
        
        # Masses
        part0.create_dataset('Masses', data=ic_data['masses'])
        
        # Internal energy
        part0.create_dataset('InternalEnergy', data=ic_data['internal_energies'])
        
        # Particle IDs
        part0.create_dataset('ParticleIDs', data=ic_data['ids'])
        
        # Smoothing lengths (estimate)
        # For a regular grid, smoothing length ~ particle spacing
        h = np.full(ic_data['n_particles'], 
                   2.0 * min(ic_data['box_size']) / max(150, 75))
        part0.create_dataset('SmoothingLength', data=h)
        
    print(f"Written initial conditions to {filepath}")
    return filepath

# Write the IC file
ic_filename = 'kelvin_helmholtz.hdf5'
ic_filepath = write_swift_ic_file(ic_data, ic_filename)

## 4. Creating Parameter File for SWIFT

Now we need to create a parameter file that tells SWIFT how to run the simulation:

In [ ]:
def create_swift_parameter_file(ic_filename, output_name='kelvin_helmholtz'):
    """
    Create a SWIFT parameter file for the Kelvin-Helmholtz simulation
    """
    
    # Ensure output directory exists
    os.makedirs('../params', exist_ok=True)
    param_file = f'../params/{output_name}.yml'
    
    param_content = f"""
# Define the system of units to use internally. 
InternalUnitSystem:
  UnitMass_in_cgs:     1   # grams
  UnitLength_in_cgs:   1   # centimeters 
  UnitVelocity_in_cgs: 1   # centimeters per second
  UnitCurrent_in_cgs:  1   # amperes
  UnitTemp_in_cgs:     1   # kelvin

# Parameters governing the time integration
TimeIntegration:
  time_begin: 0.0    # The starting time of the simulation (in internal units).
  time_end:   2.0    # The end time of the simulation (in internal units).
  dt_min:     1e-6   # The minimal time-step size of the simulation (in internal units).
  dt_max:     1e-2   # The maximal time-step size of the simulation (in internal units).

# Parameters governing the snapshots
Snapshots:
  basename:            {output_name}  # Common part of the name of output files
  output_list_on:      1              # Enable the output list
  output_list:         ../params/output_list.txt # File containing the output times
  compression:         1              # GZIP compression level (0-9)

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          0.01    # Time between statistics output

# Parameters for the hydrodynamics scheme
SPH:
  resolution_eta:        1.2348   # Target smoothing length in units of the mean inter-particle separation
  h_min_ratio:           0.1      # Minimal smoothing length in units of the softening
  h_max:                 10.0     # Maximal smoothing length in internal units
  CFL_condition:         0.2      # Courant-Friedrichs-Lewy condition for time integration
  minimal_temperature:   10.0     # Minimal temperature in internal units

# Parameters related to the initial conditions
InitialConditions:
  file_name:  ../ics/{ic_filename}    # The file to read
  periodic:   1                       # Periodic boundary conditions

# Dimensionality of the problem
MetaData:
  run_name:   {output_name}

# Parameters for the GIZMO-MFV hydro scheme
GIZMOMFV:
  eta:                   1.2348   # Smoothing length parameter
  beta:                  1.5      # Volume definition parameter

# Equation of state
EoS:
  isothermal_internal_energy: 1.0   # Isothermal equation of state
"""
    
    with open(param_file, 'w') as f:
        f.write(param_content)
    
    # Create output list file
    output_list_file = '../params/output_list.txt'
    output_times = np.linspace(0, 2.0, 41)  # 41 outputs from t=0 to t=2
    with open(output_list_file, 'w') as f:
        for t in output_times:
            f.write(f"{t:.3f}\n")
    
    print(f"Created parameter file: {param_file}")
    print(f"Created output list: {output_list_file}")
    return param_file

# Create parameter file
param_file = create_swift_parameter_file(ic_filename)

## 5. Running the Simulation

If you have SWIFT installed, you can run the simulation. Otherwise, we'll load pre-computed results.

In [ ]:
def run_swift_simulation(param_file, swift_executable='swift'):
    """
    Run SWIFT simulation if executable is available
    """
    try:
        # Check if SWIFT is available
        result = subprocess.run(['which', swift_executable], 
                               capture_output=True, text=True)
        if result.returncode != 0:
            print(f"SWIFT executable '{swift_executable}' not found.")
            print("Please install SWIFT or provide the correct path.")
            return False
        
        print(f"Found SWIFT at: {result.stdout.strip()}")
        
        # Change to snapshots directory for output
        os.makedirs('../snapshots', exist_ok=True)
        original_dir = os.getcwd()
        os.chdir('../snapshots')
        
        # Run SWIFT
        cmd = [swift_executable, '--hydro', param_file]
        print(f"Running: {' '.join(cmd)}")
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        os.chdir(original_dir)
        
        if result.returncode == 0:
            print("Simulation completed successfully!")
            return True
        else:
            print(f"Simulation failed with return code {result.returncode}")
            print(f"Error output: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print("Simulation timed out (5 minutes)")
        return False
    except Exception as e:
        print(f"Error running simulation: {e}")
        return False

# Try to run the simulation
simulation_success = run_swift_simulation(param_file)

if not simulation_success:
    print("\nNo worries! You can still explore the initial conditions and")
    print("analysis techniques. To run simulations, install SWIFT following")
    print("the instructions in the README.")

## 6. Analysis and Visualization

Let's create some analysis functions that work whether or not you've run the simulation:

In [ ]:
def load_swift_snapshot(filename):
    """
    Load a SWIFT snapshot from HDF5 file
    """
    try:
        with h5py.File(filename, 'r') as f:
            # Read header
            header = f['Header']
            time = header.attrs['Time']
            box_size = header.attrs['BoxSize']
            
            # Read particle data
            part0 = f['PartType0']
            
            data = {
                'time': time,
                'box_size': box_size,
                'positions': part0['Coordinates'][:],
                'velocities': part0['Velocities'][:],
                'densities': part0['Density'][:] if 'Density' in part0 else None,
                'internal_energy': part0['InternalEnergy'][:],
                'smoothing_length': part0['SmoothingLength'][:],
                'masses': part0['Masses'][:]
            }
            
        return data
    except Exception as e:
        print(f"Error loading snapshot {filename}: {e}")
        return None

def plot_snapshot(data, title="", save_path=None):
    """
    Create a visualization of a snapshot
    """
    if data is None:
        print("No data to plot")
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    pos = data['positions']
    vel = data['velocities']
    
    # Density plot
    if data['densities'] is not None:
        im1 = axes[0].scatter(pos[:,0], pos[:,1], c=data['densities'], 
                             s=2, cmap='viridis', vmin=0.5, vmax=2.0)
        axes[0].set_title(f'Density (t={data["time"]:.2f})')
        plt.colorbar(im1, ax=axes[0])
    else:
        axes[0].scatter(pos[:,0], pos[:,1], s=1, alpha=0.6)
        axes[0].set_title(f'Particle positions (t={data["time"]:.2f})')
    
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_aspect('equal')
    
    # X-velocity
    im2 = axes[1].scatter(pos[:,0], pos[:,1], c=vel[:,0], 
                         s=2, cmap='RdBu_r', vmin=-1.5, vmax=1.5)
    axes[1].set_title('X-Velocity')
    axes[1].set_xlabel('x') 
    axes[1].set_ylabel('y')
    axes[1].set_aspect('equal')
    plt.colorbar(im2, ax=axes[1])
    
    # Vorticity (curl of velocity)
    # For 2D: ω = ∂v_x/∂y - ∂v_y/∂x
    # We'll approximate this using local finite differences
    try:
        # Simple approximation of vorticity
        vel_mag = np.sqrt(vel[:,0]**2 + vel[:,1]**2)
        im3 = axes[2].scatter(pos[:,0], pos[:,1], c=vel_mag, 
                             s=2, cmap='plasma')
        axes[2].set_title('Velocity Magnitude')
        plt.colorbar(im3, ax=axes[2])
    except:
        axes[2].scatter(pos[:,0], pos[:,1], c=vel[:,1], 
                       s=2, cmap='RdBu_r')
        axes[2].set_title('Y-Velocity')
    
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y') 
    axes[2].set_aspect('equal')
    
    if title:
        fig.suptitle(title, fontsize=16)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
    
    plt.show()

# Test with initial conditions
print("Initial conditions visualization:")
# Convert our IC data to the format expected by plot_snapshot
ic_plot_data = {
    'time': 0.0,
    'box_size': ic_data['box_size'],
    'positions': ic_data['positions'],
    'velocities': ic_data['velocities'],
    'densities': ic_data['densities'],
    'internal_energy': ic_data['internal_energies'],
    'smoothing_length': None,
    'masses': ic_data['masses']
}

plot_snapshot(ic_plot_data, "Initial Conditions")

## 7. Loading and Analyzing Simulation Results

If the simulation ran successfully, let's load and analyze the results:

In [ ]:
import glob

# Look for snapshot files
snapshot_files = sorted(glob.glob('../snapshots/kelvin_helmholtz_*.hdf5'))

if snapshot_files:
    print(f"Found {len(snapshot_files)} snapshot files")
    
    # Load and plot a few snapshots
    snapshot_indices = [0, len(snapshot_files)//4, len(snapshot_files)//2, -1]
    
    for i in snapshot_indices:
        if i < len(snapshot_files):
            print(f"\nLoading snapshot {i}: {snapshot_files[i]}")
            snapshot_data = load_swift_snapshot(snapshot_files[i])
            if snapshot_data:
                plot_snapshot(snapshot_data, 
                             f"Kelvin-Helmholtz Instability Evolution")
else:
    print("No snapshot files found. This is expected if SWIFT simulation didn't run.")
    print("The initial conditions and analysis framework are still useful!")

## 8. Creating Movies (if simulation data exists)

Let's create a simple movie-making function:

In [ ]:
def create_movie_frames(snapshot_files, output_dir='../videos'):
    """
    Create movie frames from snapshot files
    """
    os.makedirs(output_dir, exist_ok=True)
    
    frame_files = []
    
    for i, snapshot_file in enumerate(snapshot_files):
        print(f"Processing frame {i+1}/{len(snapshot_files)}", end='\r')
        
        data = load_swift_snapshot(snapshot_file)
        if data is None:
            continue
            
        frame_file = f"{output_dir}/frame_{i:04d}.png"
        plot_snapshot(data, 
                     f"Kelvin-Helmholtz Instability (t={data['time']:.2f})",
                     save_path=frame_file)
        plt.close()  # Close the figure to save memory
        
        frame_files.append(frame_file)
    
    print(f"\nCreated {len(frame_files)} frames in {output_dir}")
    
    # Try to create MP4 movie using ffmpeg if available
    try:
        movie_file = f"{output_dir}/kelvin_helmholtz_movie.mp4"
        cmd = ['ffmpeg', '-y', '-framerate', '10', 
               '-i', f'{output_dir}/frame_%04d.png',
               '-c:v', 'libx264', '-pix_fmt', 'yuv420p',
               movie_file]
        
        result = subprocess.run(cmd, capture_output=True)
        if result.returncode == 0:
            print(f"Created movie: {movie_file}")
        else:
            print("ffmpeg not available. Individual frames saved.")
    except:
        print("ffmpeg not available. Individual frames saved.")
    
    return frame_files

# Create movie if we have snapshots
if snapshot_files and len(snapshot_files) > 5:
    print("Creating movie frames...")
    create_movie_frames(snapshot_files)
else:
    print("Not enough snapshots for movie creation.")

## 9. Parameter Exploration

Let's explore how different parameters affect the initial conditions:

In [ ]:
# Create a parameter study
print("Parameter exploration: How do different settings affect the setup?")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Different parameter combinations
param_sets = [
    {'velocity_shear': 1.0, 'density_contrast': 1.0, 'perturbation_amplitude': 0.05},
    {'velocity_shear': 2.0, 'density_contrast': 1.0, 'perturbation_amplitude': 0.05},
    {'velocity_shear': 1.0, 'density_contrast': 3.0, 'perturbation_amplitude': 0.05},
    {'velocity_shear': 1.0, 'density_contrast': 1.0, 'perturbation_amplitude': 0.2},
    {'velocity_shear': 3.0, 'density_contrast': 2.0, 'perturbation_amplitude': 0.1},
    {'velocity_shear': 1.0, 'density_contrast': 0.5, 'perturbation_amplitude': 0.1},
]

titles = [
    'Low shear, no density contrast',
    'High shear, no density contrast', 
    'Low shear, high density contrast',
    'High perturbation amplitude',
    'High shear + density contrast',
    'Inverted density contrast'
]

for i, (params, title) in enumerate(zip(param_sets, titles)):
    # Generate IC with these parameters
    test_ic = generate_kelvin_helmholtz_ic(n_particles_x=100, n_particles_y=50,
                                          box_size=[4.0, 2.0], **params)
    
    # Plot density field
    pos = test_ic['positions']
    vel = test_ic['velocities']
    rho = test_ic['densities']
    
    im = axes[i].scatter(pos[:,0], pos[:,1], c=rho, s=2, cmap='viridis')
    
    # Add velocity arrows
    skip = 15
    axes[i].quiver(pos[::skip,0], pos[::skip,1], 
                  vel[::skip,0], vel[::skip,1], 
                  scale=15, alpha=0.7, color='red', width=0.003)
    
    axes[i].set_title(title, fontsize=10)
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('y')
    axes[i].set_aspect('equal')

plt.tight_layout()
plt.suptitle('Parameter Study: Effect on Initial Conditions', fontsize=16, y=1.02)
plt.show()

## Summary and Next Steps

In this notebook, we've:

1. **Learned about the Kelvin-Helmholtz instability** - its physical origin and importance
2. **Generated initial conditions** with two fluid layers and velocity shear
3. **Created SWIFT-compatible files** for running SPH simulations
4. **Set up parameter files** for the simulation
5. **Developed analysis and visualization tools**
6. **Explored parameter variations** and their effects

### Key Physics Insights:
- The Kelvin-Helmholtz instability grows from small perturbations in shear flows
- Density contrasts can stabilize or destabilize the interface
- The characteristic "cat's eye" structures develop from the velocity shear

### Next Steps:
1. **Run the simulation** with SWIFT if you have it installed
2. **Experiment with different parameters** - try higher density contrasts, different perturbation wavelengths
3. **Add physical effects** - viscosity, cooling, magnetic fields
4. **Move to 3D** - the instability has rich 3D structure

### Challenges to Try:
- Implement multi-wavelength perturbations
- Add a background flow
- Try different density profiles (gradual vs sharp interfaces)
- Compare with linear stability theory predictions

The Kelvin-Helmholtz instability is fundamental to many astrophysical phenomena, from planetary atmospheres to accretion disk flows. Understanding its behavior in SPH simulations is crucial for modeling real astrophysical systems!